# Imports

In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ast

# Load Data

In [9]:
df = pd.read_csv("Data/cars.csv")

# Clean Data

## Drop Duplicates

In [10]:
df = df.loc[df.id!='[]'].drop_duplicates(subset='id')

In [11]:
df.shape

(15932, 6)

## Prepare Columns

#### Id

In [12]:
df['id'] = df['id'].str[4:-2].astype(int)
df.rename(columns={'id': 'Id'}, inplace=True)
df.set_index('Id', inplace=True)

In [13]:
df.sample(2)

,location,price,specifications-1,specifications-2,stickers
Id,,,,,
95572074,"['თბილისი', '67ნახვა2 დღე']","['18,825']","[['მწარმოებელი', 'Volkswagen'], ['მოდელი', 'Pa...","[['ჰიდრავლიკა', 'Yes'], ['ბორტკომპიუტერი', 'Ye...",['']
95464782,"['ბათუმი', '44ნახვა2 დღე']","['8,618']","[['მწარმოებელი', 'Toyota'], ['მოდელი', 'Prius'...","[['ჰიდრავლიკა', 'Yes'], ['ბორტკომპიუტერი', 'No...",['']


### Location

In [14]:
df['location'] = df['location'].apply(ast.literal_eval).str[0]
df.rename(columns={'location': 'Location'}, inplace=True)

In [15]:
df['Location'].value_counts()

Location
თბილისი                  8899
რუსთავის ავტობაზრობა     1409
რუსთავი                  1053
ბათუმი                   1030
კავკასიის ავტომარკეტი     728
ქუთაისი                   596
გორი                      571
ფოთი                      151
ზესტაფონი                 127
ზუგდიდი                   124
თელავი                    123
მარნეული                  107
ახალციხე                   93
ხაშური                     85
კასპი                      74
მცხეთა                     66
გურჯაანი                   53
ახალქალაქი                 45
ქობულეთი                   43
ქარელი                     42
ოზურგეთი                   39
ბორჯომი                    38
სენაკი                     37
საგარეჯო                   33
ფოთის "გეზ"-ი              32
სამტრედია                  29
სიღნაღი                    29
ლაგოდეხი                   28
ჭიათურა                    24
ყვარელი                    21
საჩხერე                    21
ახმეტა                     19
ბოლნისი                    18
დ

### Price

In [16]:
df['price'] = df['price'].str[2:-2].str.replace(',', '').astype(int)
df.rename(columns={'price': 'Price'}, inplace=True)

In [17]:
# Set pandas display option to show floats without scientific notation
pd.set_option('display.float_format', lambda x: '{:.5f}'.format(x))

df['Price'].describe()

count     15932.00000
mean      33645.74718
std       70799.74714
min           1.00000
25%       13058.00000
50%       24811.00000
75%       41787.00000
max     7777780.00000
Name: Price, dtype: float64

### specifications-1

In [18]:
df['specifications-1']

Id
93284442    [['მწარმოებელი', 'Mercedes-Benz'], ['მოდელი', ...
94416194    [['მწარმოებელი', 'BMW'], ['მოდელი', '428'], ['...
95335570    [['მწარმოებელი', 'Toyota'], ['მოდელი', 'RAV 4'...
95470114    [['მწარმოებელი', 'Mercedes-Benz'], ['მოდელი', ...
85990507    [['მწარმოებელი', 'Toyota'], ['მოდელი', 'Prius'...
                                  ...                        
95698770    [['მწარმოებელი', 'Volkswagen'], ['მოდელი', 'Ti...
92268310    [['მწარმოებელი', 'Volkswagen'], ['მოდელი', 'Pa...
95695366    [['მწარმოებელი', 'Volkswagen'], ['მოდელი', 'CC...
95698710    [['მწარმოებელი', 'Volkswagen'], ['მოდელი', 'CC...
95699122    [['მწარმოებელი', 'Volkswagen'], ['მოდელი', 'Je...
Name: specifications-1, Length: 15932, dtype: object

In [19]:
# Turn string of list into a python list
df['specifications-1'] = df['specifications-1'].apply(lambda x: ast.literal_eval(x))

In [20]:
# Function to find the index of the set containing the specification
def find_index_of_set_with_spec(set_list, spec):
    for i, s in enumerate(set_list):
        if spec in s:
            return round(i)
    return None

In [21]:
# Extract Specs from each set
def extract_specs(s, spec_name):
    if pd.notna(s):
        return [spec for spec in s if not spec.startswith(spec_name)][0]
    else:
        return pd.NA

#### Manufacturer

In [22]:
# Find the index of the set containing the Manufacturer name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'მწარმოებელი'))
df['Index_of_set'].unique()

array([0], dtype=int64)

In [23]:
df['Manufacturer'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Manufacturer'] = df['Manufacturer'].str[1]

# Manufacturer description table
df['Manufacturer'].value_counts()

Manufacturer
Toyota           2469
Mercedes-Benz    1663
Nissan           1109
Subaru           1082
BMW              1077
Honda            1029
Hyundai          1028
Mitsubishi       1025
Mazda            1003
Volkswagen        918
Lexus             905
Kia               903
Ford              865
Chevrolet         856
Name: count, dtype: int64

#### Model

In [24]:
# Find the index of the set containing the Model name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'მოდელი'))
df['Index_of_set'].unique()

array([1], dtype=int64)

In [25]:
df['Model'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Model'] = df['Model'].str[1]

# Model description table
df['Model'].value_counts()

Model
Camry       680
FIT         555
Forester    506
RAV 4       440
CX-5        437
           ... 
SLC 300       1
GLE 450       1
C 280         1
Maxima        1
Scirocco      1
Name: count, Length: 588, dtype: int64

#### Year

In [26]:
# Find the index of the set containing the Year name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'წელი'))
df['Index_of_set'].unique()

array([2], dtype=int64)

In [27]:
df['Year'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Year'] = df['Year'].str[1]

# Year description table
df['Year'].value_counts()

Year
2019    1279
2017    1250
2018    1226
2015    1091
2014    1078
2013    1066
2016    1058
2020     941
2012     912
2021     638
2005     578
2011     559
2022     536
2003     397
2004     385
2006     353
2010     351
2007     326
2008     302
2002     283
2001     213
2000     202
2009     190
1999     147
2023     134
1998     119
1997      97
1996      84
1995      45
1994      36
1992      13
1993      12
1991       9
1990       4
1989       4
1984       3
1979       2
1986       2
1988       2
1974       1
1985       1
1971       1
1983       1
1943       1
Name: count, dtype: int64

#### Category

In [28]:
# Find the index of the set containing the Category name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'კატეგორია'))
df['Index_of_set'].unique()

array([3], dtype=int64)

In [29]:
df['Category'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Category'] = df['Category'].str[1]

# Category description table
df['Category'].value_counts()

Category
სედანი        6326
ჯიპი          5830
ჰეტჩბექი      2229
მინივენი       472
კუპე           340
ფურგონი        261
უნივერსალი     255
პიკაპი          95
კაბრიოლეტი      63
კროსოვერი       61
Name: count, dtype: int64

#### Mileage

In [30]:
# Find the index of the set containing the Mileage name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'გარბენი'))
df['Index_of_set'].unique()

array([4], dtype=int64)

In [31]:
df['Mileage'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Mileage'] = df['Mileage'].str[1].str[:-3].astype(int)

# Set the float_format option to display one decimal point
pd.options.display.float_format = '{:.5f}'.format

df['Mileage'].describe()

count     15932.00000
mean     126076.14844
std       97743.53608
min           0.00000
25%       52300.00000
50%      115000.00000
75%      180000.00000
max     1554008.00000
Name: Mileage, dtype: float64

#### Fuel type

In [32]:
# Find the index of the set containing the Fuel type name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'საწვავის ტიპი'))
df['Index_of_set'].unique()

array([5], dtype=int64)

In [33]:
df['Fuel type'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Fuel type'] = df['Fuel type'].str[1]

# Fuel type description table
df['Fuel type'].value_counts()

Fuel type
ბენზინი              11733
ჰიბრიდი               1824
დიზელი                1204
ბუნებრივი გაზი         540
თხევადი გაზი           296
დატენვადი ჰიბრიდი      212
ელექტრო                123
Name: count, dtype: int64

#### Engine Volume

In [34]:
# Find the index of the set containing the Engine Volume name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'ძრავის მოცულობა'))
df['Index_of_set'].unique()

array([6], dtype=int64)

In [35]:
df['Engine Volume'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Engine Volume'] = df['Engine Volume'].str[1]

# Engine Volume description table
df['Engine Volume'].value_counts()

Engine Volume
2.5          2947
2.0          2434
2.4          1127
1.5          1088
1.8           962
             ... 
6.2 turbo       1
5.2             1
6.3 turbo       1
4.3 turbo       1
5.6             1
Name: count, Length: 99, dtype: int64

#### Cylinders

In [36]:
df['Cylinders'] = df['specifications-1'].str[7].str[1]
df['Cylinders'].value_counts()

Cylinders
4             11938
6              2691
8               675
5               220
3               125
1               111
12               86
2                46
10               17
9                 8
7                 7
ვარიატორი         5
11                2
ავტომატიკა        1
Name: count, dtype: int64

In [37]:
df[(df['Cylinders']=='ვარიატორი') | (df['Cylinders']=='ავტომატიკა')]

,Location,Price,specifications-1,specifications-2,stickers,Index_of_set,Manufacturer,Model,Year,Category,Mileage,Fuel type,Engine Volume,Cylinders
Id,,,,,,,,,,,,,,
77714573,რუსთავის ავტობაზრობა,35257,"[[მწარმოებელი, BMW], [მოდელი, I3], [წელი, 2014...","[['ჰიდრავლიკა', 'Yes'], ['ბორტკომპიუტერი', 'Ye...",[''],6,BMW,I3,2014,ჰეტჩბექი,22257,ელექტრო,0.0,ვარიატორი
79780065,რუსთავის ავტობაზრობა,39959,"[[მწარმოებელი, Nissan], [მოდელი, Leaf], [წელი,...","[['ჰიდრავლიკა', 'Yes'], ['ბორტკომპიუტერი', 'Ye...",[''],6,Nissan,Leaf,2018,ჰეტჩბექი,40847,ელექტრო,0.0,ავტომატიკა
82848299,რუსთავი,14364,"[[მწარმოებელი, Mitsubishi], [მოდელი, i-Miev], ...","[['ჰიდრავლიკა', 'No'], ['ბორტკომპიუტერი', 'Yes...",[''],6,Mitsubishi,i-Miev,2013,ჰეტჩბექი,120000,ელექტრო,0.0,ვარიატორი
82839947,რუსთავი,40481,"[[მწარმოებელი, Nissan], [მოდელი, Leaf], [წელი,...","[['ჰიდრავლიკა', 'Yes'], ['ბორტკომპიუტერი', 'Ye...",[''],6,Nissan,Leaf,2017,სედანი,120000,ელექტრო,0.0,ვარიატორი
77715793,რუსთავის ავტობაზრობა,39436,"[[მწარმოებელი, BMW], [მოდელი, I3], [წელი, 2016...","[['ჰიდრავლიკა', 'Yes'], ['ბორტკომპიუტერი', 'Ye...",[''],6,BMW,I3,2016,ჰეტჩბექი,56536,ელექტრო,0.0,ვარიატორი
77777357,რუსთავის ავტობაზრობა,24288,"[[მწარმოებელი, BMW], [მოდელი, I3], [წელი, 2014...","[['ჰიდრავლიკა', 'Yes'], ['ბორტკომპიუტერი', 'Ye...",[''],6,BMW,I3,2014,ჰეტჩბექი,58328,ელექტრო,0.0,ვარიატორი


In [38]:
df.loc[(df['Cylinders']=='ვარიატორი') | (df['Cylinders']=='ავტომატიკა'), 'Cylinders'] = np.nan

In [39]:
df.Cylinders = df.Cylinders.astype('Int64')
df['Cylinders'].value_counts()

Cylinders
4     11938
6      2691
8       675
5       220
3       125
1       111
12       86
2        46
10       17
9         8
7         7
11        2
Name: count, dtype: Int64

#### Gear box type

In [40]:
# Find the index of the set containing the Gear box type name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'გადაცემათა კოლოფი'))
df['Index_of_set'].unique()

array([8, 7], dtype=int64)

In [41]:
df['Gear box type'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Gear box type'] = df['Gear box type'].str[1]

# Gear box type description table
df['Gear box type'].value_counts()

Gear box type
ავტომატიკა    6669
ტიპტრონიკი    6367
ვარიატორი     1449
მექანიკა      1447
Name: count, dtype: int64

#### Drive wheels

In [42]:
# Find the index of the set containing the Drive wheels name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'წამყვანი თვლები'))
df['Index_of_set'].unique()

array([ 9.,  8., nan])

In [43]:
df['Drive wheels'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Drive wheels'] = df['Drive wheels'].str[1]

# Drive wheels description table
df['Drive wheels'].value_counts()

Drive wheels
წინა     8311
4x4      5730
უკანა    1885
Name: count, dtype: int64

#### Doors

In [44]:
# Find the index of the set containing the Doors name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'კარები'))
df['Index_of_set'].unique()

array([10.,  9., nan])

In [45]:
df['Doors'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Doors'] = df['Doors'].str[1]

# Doors description table
df.Doors.value_counts()

Doors
4/5    14739
2/3      691
>5       500
Name: count, dtype: int64

#### Airbags

In [46]:
# Find the index of the set containing the Airbags name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'აირბეგი'))
df['Index_of_set'].unique()

array([11., nan, 10.,  9.])

In [47]:
df['Airbags'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Airbags'] = df['Airbags'].str[1].astype('Int64')

# Airbags description table
df.Airbags.value_counts()

Airbags
12    3830
9     2961
8     2710
6     1602
4     1514
10    1336
2      786
5      289
7      182
3       70
11      65
1       56
Name: count, dtype: Int64

#### Wheel

In [48]:
# Find the index of the set containing the Wheel name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'საჭე'))
df['Index_of_set'].unique()

array([12, 11, 10], dtype=int64)

In [49]:
df['Wheel'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Wheel'] = df['Wheel'].str[1]

# Wheel description table
df.Wheel.value_counts()

Wheel
მარცხენა    13846
მარჯვენა     2086
Name: count, dtype: int64

#### Color

In [50]:
# Find the index of the set containing the Color name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'ფერი'))
df['Index_of_set'].unique()

array([13., 12., nan, 11.])

In [51]:
df['Color'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Color'] = df['Color'].str[1]

# Color description table
df.Color.value_counts()

Color
შავი            4196
თეთრი           2991
ვერცხლისფერი    2639
რუხი            2628
ლურჯი           1505
წითელი           651
შინდისფერი       356
მწვანე           214
ყავისფერი        204
ცისფერი          193
ოქროსფერი        129
ჩალისფერი         81
ნარინჯისფერი      60
ყვითელი           38
იისფერი           38
ვარდისფერი         6
Name: count, dtype: int64

#### Interior color

In [52]:
# Find the index of the set containing the Interior color name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'სალონის ფერი'))
df['Index_of_set'].unique()

array([14., 13., 12., nan])

In [53]:
df['Interior color'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Interior color'] = df['Interior color'].str[1]

# Interior color description table
df['Interior color'].value_counts()

Interior color
შავი            10081
რუხი             2711
ჩალისფერი        1368
თეთრი             730
ყავისფერი         378
წითელი            213
ლურჯი             100
ყვითელი            94
ოქროსფერი          89
ნარინჯისფერი       87
შინდისფერი         80
Name: count, dtype: int64

#### Interior material

In [54]:
# Find the index of the set containing the Interior material name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'სალონის მასალა'))
df['Index_of_set'].unique()

array([15., 14., 13., nan])

In [55]:
df['Interior material'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Interior material'] = df['Interior material'].str[1]

# Interior material description table
df['Interior material'].value_counts()

Interior material
ტყავი              7259
ნაჭერი             7114
კომბინირებული      1146
ალკანტარა           326
ხელოვნური ტყავი      86
Name: count, dtype: int64

#### Exchange

In [56]:
# Find the index of the set containing the Exchange name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'გაცვლა'))
df['Index_of_set'].unique()

array([16, 15, 14], dtype=int64)

In [57]:
df['Exchange'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Exchange'] = df['Exchange'].str[1]

# Exchange description table
df.Exchange.value_counts()

Exchange
არა    15420
კი       512
Name: count, dtype: int64

#### Technical inspection

In [58]:
# Find the index of the set containing the Technical inspection name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'ტექ. დათვალიერება'))
df['Index_of_set'].unique()

array([17, 16, 15], dtype=int64)

In [59]:
df['Technical inspection'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Technical inspection'] = df['Technical inspection'].str[1]

# Technical inspection description table
df['Technical inspection'].value_counts()

Technical inspection
კი     15395
არა      537
Name: count, dtype: int64

#### Catalyst

In [60]:
# Find the index of the set containing the Catalyst name for each row
df['Index_of_set'] = df['specifications-1'].apply(lambda x: find_index_of_set_with_spec(x, 'კატალიზატორი'))
df['Index_of_set'].unique()

array([18., nan, 17., 16.])

In [61]:
df['Catalyst'] = df.apply(lambda row: row['specifications-1'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Catalyst'] = df['Catalyst'].str[1]

# Catalyst description table
df.Catalyst.value_counts()

Catalyst
კი     12350
არა     1808
Name: count, dtype: int64

### specifications-2

In [62]:
# Turn string of list into a python list
df['specifications-2'] = df['specifications-2'].apply(lambda x: ast.literal_eval(x))

In [63]:
df['specifications-2']

Id
93284442    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
94416194    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
95335570    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
95470114    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
85990507    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
                                  ...                        
95698770    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
92268310    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
95695366    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
95698710    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, Yes], [კო...
95699122    [[ჰიდრავლიკა, Yes], [ბორტკომპიუტერი, No], [კონ...
Name: specifications-2, Length: 15932, dtype: object

#### Steering Hydraulics

In [64]:
# Find the index of the set containing the ჰიდრავლიკა name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'ჰიდრავლიკა'))
df['Index_of_set'].unique()

array([0], dtype=int64)

In [65]:
df['Steering Hydraulics'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Steering Hydraulics'] = df['Steering Hydraulics'].str[1]

# Catalyst description table
df['Steering Hydraulics'].value_counts()

Steering Hydraulics
Yes    14573
No      1359
Name: count, dtype: int64

#### On-Board Computer

In [66]:
# Find the index of the set containing the ბორტკომპიუტერი name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'ბორტკომპიუტერი'))
df['Index_of_set'].unique()

array([1], dtype=int64)

In [67]:
df['On-Board Computer'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['On-Board Computer'] = df['On-Board Computer'].str[1]

# ბორტკომპიუტერი description table
df['On-Board Computer'].value_counts()

On-Board Computer
Yes    12596
No      3336
Name: count, dtype: int64

#### Air Conditioning

In [68]:
# Find the index of the set containing the კონდიციონერი name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'კონდიციონერი'))
df['Index_of_set'].unique()

array([2], dtype=int64)

In [69]:
df['Air Conditioning'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Air Conditioning'] = df['Air Conditioning'].str[1]

# კონდიციონერი description table
df['Air Conditioning'].value_counts()

Air Conditioning
Yes    14397
No      1535
Name: count, dtype: int64

#### Parking Control

In [70]:
# Find the index of the set containing the პარკინგკონტროლი name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'პარკინგკონტროლი'))
df['Index_of_set'].unique()

array([3], dtype=int64)

In [71]:
df['Parking Control'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Parking Control'] = df['Parking Control'].str[1]

# პარკინგკონტროლი description table
df['Parking Control'].value_counts()

Parking Control
Yes    9328
No     6604
Name: count, dtype: int64

#### Rear View Camera

In [72]:
# Find the index of the set containing the უკანა ხედვის კამერა name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'უკანა ხედვის კამერა'))
df['Index_of_set'].unique()

array([4], dtype=int64)

In [73]:
df['Rear View Camera'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Rear View Camera'] = df['Rear View Camera'].str[1]

# უკანა ხედვის კამერა description table
df['Rear View Camera'].value_counts()

Rear View Camera
Yes    11204
No      4728
Name: count, dtype: int64

#### Electric Side Mirros

In [74]:
# Find the index of the set containing the ელექტრო შუშები name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'ელექტრო შუშები'))
df['Index_of_set'].unique()

array([5], dtype=int64)

In [75]:
df['Electric Side Mirros'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Electric Side Mirros'] = df['Electric Side Mirros'].str[1]

# ელექტრო შუშები description table
df['Electric Side Mirros'].value_counts()

Electric Side Mirros
Yes    14550
No      1382
Name: count, dtype: int64

#### Climate Control

In [76]:
# Find the index of the set containing the კლიმატკონტროლი name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'კლიმატკონტროლი'))
df['Index_of_set'].unique()

array([6], dtype=int64)

In [77]:
df['Climate Control'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Climate Control'] = df['Climate Control'].str[1]

# კლიმატკონტროლი description table
df['Climate Control'].value_counts()

Climate Control
Yes    12921
No      3011
Name: count, dtype: int64

#### Cruise Control

In [78]:
# Find the index of the set containing the კრუიზ-კონტროლი name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'კრუიზ-კონტროლი'))
df['Index_of_set'].unique()

array([7], dtype=int64)

In [79]:
df['Cruise Control'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Cruise Control'] = df['Cruise Control'].str[1]

# კრუიზ-კონტროლი description table
df['Cruise Control'].value_counts()

Cruise Control
Yes    11700
No      4232
Name: count, dtype: int64

#### Start-Stop System

In [80]:
# Find the index of the set containing the Start/Stop სისტემა name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'Start/Stop სისტემა'))
df['Index_of_set'].unique()

array([8], dtype=int64)

In [81]:
df['Start-Stop System'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Start-Stop System'] = df['Start-Stop System'].str[1]

# Start/Stop სისტემა description table
df['Start-Stop System'].value_counts()

Start-Stop System
Yes    8858
No     7074
Name: count, dtype: int64

#### Sunroof

In [82]:
# Find the index of the set containing the ლუქი name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'ლუქი'))
df['Index_of_set'].unique()

array([9], dtype=int64)

In [83]:
df['Sunroof'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Sunroof'] = df['Sunroof'].str[1]

# ლუქი description table
df['Sunroof'].value_counts()

Sunroof
No     8817
Yes    7115
Name: count, dtype: int64

#### Heated Seats

In [84]:
# Find the index of the set containing the სავარძლის გათბობა name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'სავარძლის გათბობა'))
df['Index_of_set'].unique()

array([10], dtype=int64)

In [85]:
df['Heated Seats'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Heated Seats'] = df['Heated Seats'].str[1]

# სავარძლის გათბობა description table
df['Heated Seats'].value_counts()

Heated Seats
Yes    9795
No     6137
Name: count, dtype: int64

#### Memory Seats

In [86]:
# Find the index of the set containing the სავარძლის მეხსიერება name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'სავარძლის მეხსიერება'))
df['Index_of_set'].unique()

array([11], dtype=int64)

In [87]:
df['Memory Seats'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Memory Seats'] = df['Memory Seats'].str[1]

# სავარძლის მეხსიერება description table
df['Memory Seats'].value_counts()

Memory Seats
Yes    8341
No     7591
Name: count, dtype: int64

#### ABS

In [88]:
# Find the index of the set containing the ABS name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'ABS'))
df['Index_of_set'].unique()

array([12], dtype=int64)

In [89]:
df['ABS'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['ABS'] = df['ABS'].str[1]

# ABS description table
df['ABS'].value_counts()

ABS
Yes    14122
No      1810
Name: count, dtype: int64

#### ESP

In [90]:
# Find the index of the set containing the მოცურების საწინააღმდეგო სისტემა name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'მოცურების საწინააღმდეგო სისტემა'))
df['Index_of_set'].unique()

array([13], dtype=int64)

In [91]:
df['ESP'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['ESP'] = df['ESP'].str[1]

# მოცურების საწინააღმდეგო სისტემა description table
df['ESP'].value_counts()

ESP
Yes    13470
No      2462
Name: count, dtype: int64

#### Central Locking

In [92]:
# Find the index of the set containing the ცენტრალური საკეტი name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'ცენტრალური საკეტი'))
df['Index_of_set'].unique()

array([14], dtype=int64)

In [93]:
df['Central Locking'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Central Locking'] = df['Central Locking'].str[1]

# ცენტრალური საკეტი description table
df['Central Locking'].value_counts()

Central Locking
Yes    14494
No      1438
Name: count, dtype: int64

#### Alarm System

In [94]:
# Find the index of the set containing the სიგნალიზაცია name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'სიგნალიზაცია'))
df['Index_of_set'].unique()

array([15], dtype=int64)

In [95]:
df['Alarm System'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Alarm System'] = df['Alarm System'].str[1]

# სიგნალიზაცია description table
df['Alarm System'].value_counts()

Alarm System
Yes    12424
No      3508
Name: count, dtype: int64

#### Fog Lamp

In [96]:
# Find the index of the set containing the სანისლე ფარები name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'სანისლე ფარები'))
df['Index_of_set'].unique()

array([16], dtype=int64)

In [97]:
df['Fog Lamp'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Fog Lamp'] = df['Fog Lamp'].str[1]

# სანისლე ფარები description table
df['Fog Lamp'].value_counts()

Fog Lamp
Yes    11996
No      3936
Name: count, dtype: int64

#### Central Screen (Navigation)

In [98]:
# Find the index of the set containing the მონიტორი (ნავიგაცია) name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'მონიტორი (ნავიგაცია)'))
df['Index_of_set'].unique()

array([17], dtype=int64)

In [99]:
df['Central Screen (Navigation)'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Central Screen (Navigation)'] = df['Central Screen (Navigation)'].str[1]

# მონიტორი (ნავიგაცია) description table
df['Central Screen (Navigation)'].value_counts()

Central Screen (Navigation)
Yes    11811
No      4121
Name: count, dtype: int64

#### AUX

In [100]:
# Find the index of the set containing the AUX name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'AUX'))
df['Index_of_set'].unique()

array([18], dtype=int64)

In [101]:
df['AUX'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['AUX'] = df['AUX'].str[1]

# AUX description table
df['AUX'].value_counts()

AUX
Yes    12911
No      3021
Name: count, dtype: int64

#### Bluetooth

In [102]:
# Find the index of the set containing the Bluetooth name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'Bluetooth'))
df['Index_of_set'].unique()

array([19], dtype=int64)

In [103]:
df['Bluetooth'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Bluetooth'] = df['Bluetooth'].str[1]

# Bluetooth description table
df['Bluetooth'].value_counts()

Bluetooth
Yes    12826
No      3106
Name: count, dtype: int64

#### Multifunction Steering Wheel

In [104]:
# Find the index of the set containing the მულტი საჭე name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'მულტი საჭე'))
df['Index_of_set'].unique()

array([20], dtype=int64)

In [105]:
df['Multifunction Steering Wheel'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Multifunction Steering Wheel'] = df['Multifunction Steering Wheel'].str[1]

# მულტი საჭე description table
df['Multifunction Steering Wheel'].value_counts()

Multifunction Steering Wheel
Yes    12271
No      3661
Name: count, dtype: int64

#### Rims

In [106]:
# Find the index of the set containing the დისკები name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'დისკები'))
df['Index_of_set'].unique()

array([21], dtype=int64)

In [107]:
df['Rims'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Rims'] = df['Rims'].str[1]

# დისკები description table
df['Rims'].value_counts()

Rims
Yes    13558
No      2374
Name: count, dtype: int64

#### Spare Tyre

In [108]:
# Find the index of the set containing the სათადარიგო საბურავი name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'სათადარიგო საბურავი'))
df['Index_of_set'].unique()

array([22], dtype=int64)

In [109]:
df['Spare Tyre'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Spare Tyre'] = df['Spare Tyre'].str[1]

# სათადარიგო საბურავი description table
df['Spare Tyre'].value_counts()

Spare Tyre
Yes    12410
No      3522
Name: count, dtype: int64

#### Didabled Accessible

In [110]:
# Find the index of the set containing the სსმპ ადაპტირებული name for each row
df['Index_of_set'] = df['specifications-2'].apply(lambda x: find_index_of_set_with_spec(x, 'სსმპ ადაპტირებული'))
df['Index_of_set'].unique()

array([23], dtype=int64)

In [111]:
df['Didabled Accessible'] = df.apply(lambda row: row['specifications-2'][int(row['Index_of_set'])] if pd.notna(row['Index_of_set']) else np.nan, axis=1)
df['Didabled Accessible'] = df['Didabled Accessible'].str[1]

# სსმპ ადაპტირებული description table
df['Didabled Accessible'].value_counts()

Didabled Accessible
No     12914
Yes     3018
Name: count, dtype: int64

### Stcikers

In [112]:
def extract_stickers(stickers, where_to_look, df):
    for sticker in stickers:
        column = df.stickers.str.contains('დაურტყმელი').astype(int).replace({0: 'No', 1: 'Yes'})
        df[sticker] = column

stickers = [
    'დაურტყმელი', 'სუფთა ისტორია', 'ახალი ჩამოყვანილი', 'ეკონომიური', 
    'ლიმიტირებული', 'შეუღებავი', 'ერთი მფლობელი', 'სასწრაფოდ', 'ევროპული', 
    'იდეალურ მდგომარეობაში', 'ოჯახისთვის', 'ლომბარდის გადაბარებით', 'მესამე რიგი'
]

extract_stickers(stickers, df.stickers, df)

In [113]:
#### drop unnecessary columns
df.drop(['specifications-1', 'specifications-2', 'stickers', 'Index_of_set'], axis=1, inplace=True)

## Translate from Georgian to English

### Column Names

In [115]:
df.columns

Index(['Location', 'Price', 'Manufacturer', 'Model', 'Year', 'Category',
       'Mileage', 'Fuel type', 'Engine Volume', 'Cylinders', 'Gear box type',
       'Drive wheels', 'Doors', 'Airbags', 'Wheel', 'Color', 'Interior color',
       'Interior material', 'Exchange', 'Technical inspection', 'Catalyst',
       'Steering Hydraulics', 'On-Board Computer', 'Air Conditioning',
       'Parking Control', 'Rear View Camera', 'Electric Side Mirros',
       'Climate Control', 'Cruise Control', 'Start-Stop System', 'Sunroof',
       'Heated Seats', 'Memory Seats', 'ABS', 'ESP', 'Central Locking',
       'Alarm System', 'Fog Lamp', 'Central Screen (Navigation)', 'AUX',
       'Bluetooth', 'Multifunction Steering Wheel', 'Rims', 'Spare Tyre',
       'Didabled Accessible', 'დაურტყმელი', 'სუფთა ისტორია',
       'ახალი ჩამოყვანილი', 'ეკონომიური', 'ლიმიტირებული', 'შეუღებავი',
       'ერთი მფლობელი', 'სასწრაფოდ', 'ევროპული', 'იდეალურ მდგომარეობაში',
       'ოჯახისთვის', 'ლომბარდის გადაბარებით', 'მესამ

In [116]:
df.columns = [
    'Location', 'Price', 'Manufacturer', 'Model', 'Year', 'Category',
    'Mileage', 'Fuel type', 'Engine Volume', 'Cylinders', 'Gear box type',
    'Drive wheels', 'Doors', 'Airbags', 'Wheel', 'Color', 'Interior color',
    'Interior material', 'Exchange', 'Technical inspection', 'Catalyst',
    'Steering Hydraulics', 'On-Board Computer', 'Air Conditioning',
    'Parking Control', 'Rear View Camera', 'Electric Side Mirros',
    'Climate Control', 'Cruise Control', 'Start-Stop System', 'Sunroof',
    'Heated Seats', 'Memory Seats', 'ABS', 'ESP', 'Central Locking',
    'Alarm System', 'Fog Lamp', 'Central Screen (Navigation)', 'AUX',
    'Bluetooth', 'Multifunction Steering Wheel', 'Rims', 'Spare Tyre',
    'Didabled Accessible', 'Undamaged', 'Clean History',
    'Newly Imported', 'Low Consumption', 'Limited edition', 'Unpainted',
    'One owner', 'Urgently', 'European', 'Perfect condition',
    'For family', 'Pawnshop', 'Third row'
    ]

In [117]:
df.columns

Index(['Location', 'Price', 'Manufacturer', 'Model', 'Year', 'Category',
       'Mileage', 'Fuel type', 'Engine Volume', 'Cylinders', 'Gear box type',
       'Drive wheels', 'Doors', 'Airbags', 'Wheel', 'Color', 'Interior color',
       'Interior material', 'Exchange', 'Technical inspection', 'Catalyst',
       'Steering Hydraulics', 'On-Board Computer', 'Air Conditioning',
       'Parking Control', 'Rear View Camera', 'Electric Side Mirros',
       'Climate Control', 'Cruise Control', 'Start-Stop System', 'Sunroof',
       'Heated Seats', 'Memory Seats', 'ABS', 'ESP', 'Central Locking',
       'Alarm System', 'Fog Lamp', 'Central Screen (Navigation)', 'AUX',
       'Bluetooth', 'Multifunction Steering Wheel', 'Rims', 'Spare Tyre',
       'Didabled Accessible', 'Undamaged', 'Clean History', 'Newly Imported',
       'Low Consumption', 'Limited edition', 'Unpainted', 'One owner',
       'Urgently', 'European', 'Perfect condition', 'For family', 'Pawnshop',
       'Third row'],
      dtyp

### Location

In [120]:
df.Location.unique()

array(['თბილისი', 'რუსთავის ავტობაზრობა', 'რუსთავი', 'ქუთაისი', 'გორი',
       'კავკასიის ავტომარკეტი', 'ბათუმი', 'ფოთი', 'სენაკი', 'ახალციხე',
       'მარნეული', 'ზუგდიდი', 'საგარეჯო', 'სამტრედია', 'ჭიათურა',
       'ხაშური', 'ქობულეთი', 'ახალქალაქი', 'მცხეთა', 'თელავი',
       'ზესტაფონი', 'ფოთის "გეზ"-ი', 'ლაგოდეხი', 'საჩხერე', 'გურჯაანი',
       'კასპი', 'ოზურგეთი', 'ქარელი', 'ტყიბული', 'ყვარელი', 'წალენჯიხა',
       'დედოფლისწყარო', 'სიღნაღი', 'ბოლნისი', 'ახმეტა', 'ბორჯომი',
       'წალკა', 'ჩხოროწყუ', 'აბაშა', 'ხობი', 'ონი', 'ხონი', 'დუშეთი',
       'ასპინძა', 'თეთრიწყარო', 'გარდაბანი', 'ხარაგაული', 'ცაგერი',
       'მარტვილი', 'სართიჭალა', 'მესტია', 'ლანჩხუთი', 'ნინოწმინდა',
       'წყალტუბო', 'ამბროლაური', 'თიანეთი', 'წეროვანი'], dtype=object)

In [125]:
mapper = {
    'თბილისი': 'Tbilisi', 'რუსთავის ავტობაზრობა': 'Rustavi Car Market', 
    'რუსთავი': 'Rustavi', 'ქუთაისი': 'Kutaisi', 'გორი': 'Gori',
    'კავკასიის ავტომარკეტი': 'Caucasus Auto Market', 'ბათუმი': 'Batumi', 
    'ფოთი': 'Poti', 'სენაკი': 'Senaki', 'ახალციხე': 'Akhaltsikhe',
    'მარნეული': 'Marneuli', 'ზუგდიდი': 'Zugdidi', 'საგარეჯო': 'Sagarejo', 
    'სამტრედია': 'Samtredia', 'ჭიათურა': 'Chiatura', 'ხაშური': 'Khashuri', 
    'ქობულეთი': 'Kobuleti', 'ახალქალაქი': 'Akhalkalaki', 'მცხეთა': 'Mtskheta', 
    'თელავი': 'Telavi', 'ზესტაფონი': 'Zestaphoni', 'ფოთის "გეზ"-ი': 'POTI "ECZ"', 
    'ლაგოდეხი': 'Lagodekhi', 'საჩხერე': 'Sachkhere', 'გურჯაანი': 'Gurjaani',
    'კასპი': 'Kaspi', 'ოზურგეთი': 'Ozurgeti', 'ქარელი': 'Kareli', 'ტყიბული': 'Tkibuli', 
    'ყვარელი': 'kvareli', 'წალენჯიხა': 'Tsalenjikha', 'დედოფლისწყარო': 'Dedoplistskaro', 
    'სიღნაღი': 'Sighnagh', 'ბოლნისი': 'Bolnisi', 'ახმეტა': 'Akhmeta', 'ბორჯომი': 'Borjomi',
    'წალკა': 'Tsalka', 'ჩხოროწყუ': 'Chkhorotsku', 'აბაშა': 'Abasha', 'ხობი': 'Khobi', 
    'ონი': 'Oni', 'ხონი': 'Khoni', 'დუშეთი': 'Dusheti', 'ასპინძა': 'Aspindza', 
    'თეთრიწყარო': 'Tetritskaro', 'გარდაბანი': 'Gardabani', 'ხარაგაული': 'Kharagauli', 
    'ცაგერი': 'Tsageri', 'მარტვილი': 'Martvili', 'სართიჭალა': 'Sartichala', 'მესტია': 'Mestia', 
    'ლანჩხუთი': 'Lanchkhuti', 'ნინოწმინდა': 'Ninotsminda', 'წყალტუბო': 'Tskaltubo', 
    'ამბროლაური': 'Ambrolauri', 'თიანეთი': 'Tianeti', 'წეროვანი': 'Tserovani'}

df.Location = df.Location.map(mapper)

### Category

In [127]:
df.Category.unique()

array(['ჯიპი', 'სედანი', 'ფურგონი', 'მინივენი', 'კუპე', 'კროსოვერი',
       'კაბრიოლეტი', 'ჰეტჩბექი', 'პიკაპი', 'უნივერსალი'], dtype=object)

In [128]:
mapper = {
    'ჯიპი': 'Jeep', 'სედანი': 'Sedan', 'ფურგონი': 'Goods wagon', 'მინივენი': 'Minivan', 
    'კუპე': 'Coupe', 'კროსოვერი': 'Crossover', 'კაბრიოლეტი': 'Cabriolet', 
    'ჰეტჩბექი': 'Hetchback', 'პიკაპი': 'Pickup', 'უნივერსალი': 'Universal'
}

df.Category = df.Category.map(mapper)

### Fuel type

In [133]:
df['Fuel type'].unique()

array(['ბენზინი', 'ჰიბრიდი', 'დიზელი', 'დატენვადი ჰიბრიდი', 'ელექტრო',
       'ბუნებრივი გაზი', 'თხევადი გაზი'], dtype=object)

In [134]:
mapper = {
    'ბენზინი': 'Petrol', 'ჰიბრიდი': 'Hybrid', 'დიზელი': 'Diesel', 'დატენვადი ჰიბრიდი': 'Plug-in Hybrid', 
    'ელექტრო': 'Electric', 'ბუნებრივი გაზი': 'CNG', 'თხევადი გაზი': 'LPG'
}

df['Fuel type'] = df['Fuel type'].map(mapper)

### Gear box type

In [136]:
df['Gear box type'].unique()

array(['ტიპტრონიკი', 'ავტომატიკა', 'ვარიატორი', 'მექანიკა'], dtype=object)

In [137]:
mapper = {
    'ტიპტრონიკი': 'Tiptronic', 'ავტომატიკა': 'Automatic', 'ვარიატორი': 'Variator', 'მექანიკა': 'Manual'
}

df['Gear box type'] = df['Gear box type'].map(mapper)

### Drive wheels

In [140]:
df['Drive wheels'].unique()

array(['4x4', 'უკანა', 'წინა', nan], dtype=object)

In [146]:
mapper = {
    '4x4': '4x4', 'უკანა': 'Rear', 'წინა': 'Front'
}

df['Drive wheels'] = df['Drive wheels'].map(mapper)

### Wheel

In [147]:
df['Wheel'].unique()

array(['მარცხენა', 'მარჯვენა'], dtype=object)

In [148]:
mapper = {
    'მარცხენა': 'Left', 'მარჯვენა': 'Right'
}

df['Wheel'] = df['Wheel'].map(mapper)

### Color

In [152]:
df['Color'].unique()

array(['შავი', 'თეთრი', 'ლურჯი', 'წითელი', 'რუხი', 'ვერცხლისფერი',
       'ოქროსფერი', 'ყვითელი', 'ყავისფერი', 'ნარინჯისფერი', 'ცისფერი',
       'შინდისფერი', 'მწვანე', 'ჩალისფერი', 'იისფერი', 'ვარდისფერი', nan],
      dtype=object)

In [153]:
mapper = {
    'შავი': 'Black', 'თეთრი': 'White', 'ლურჯი': 'Blue', 'წითელი': 'Red', 'რუხი': 'Grey', 
    'ვერცხლისფერი': 'Silver', 'ოქროსფერი': 'Golden', 'ყვითელი':' Yellow', 'ყავისფერი': 'Brown',
    'ნარინჯისფერი': 'Orange', 'ცისფერი': 'Sky Blue', 'შინდისფერი': 'Carnelian red', 
    'მწვანე': 'Green', 'ჩალისფერი': 'Beige', 'იისფერი': 'Violet', 'ვარდისფერი': 'Pink'
}

df['Color'] = df['Color'].map(mapper)

### Interior color

In [155]:
df['Interior color'].unique()

array(['შავი', 'წითელი', 'რუხი', 'ყავისფერი', 'თეთრი', 'ჩალისფერი',
       'ნარინჯისფერი', 'შინდისფერი', 'ოქროსფერი', 'ლურჯი', 'ყვითელი', nan],
      dtype=object)

In [156]:
mapper = {
    'შავი': 'Black', 'წითელი': 'Red', 'რუხი': 'Grey', 'ყავისფერი': 'Brown', 'თეთრი': 'White', 
    'ჩალისფერი': 'Beige','ნარინჯისფერი': 'Orange', 'შინდისფერი': 'Carnelian red', 
    'ოქროსფერი': 'Golden', 'ლურჯი': 'Blue', 'ყვითელი': 'Yellow'
}

df['Interior color'] = df['Interior color'].map(mapper)

### Interior material

In [158]:
df['Interior material'].unique()

array(['ტყავი', 'კომბინირებული', 'ნაჭერი', 'ალკანტარა', 'ხელოვნური ტყავი',
       nan], dtype=object)

In [159]:
mapper = {
    'ტყავი': 'Leather', 'კომბინირებული': 'Combined', 'ნაჭერი': 'Fabric', 
    'ალკანტარა': 'Alcantara', 'ხელოვნური ტყავი': 'Artificial leather'
}

df['Interior material'] = df['Interior material'].map(mapper)

### Exchange, Technical inspection, Catalyst

In [165]:
print('Exchange', ' - ', df['Exchange'].unique())
print('Technical inspection', ' - ', df['Technical inspection'].unique())
print('Catalyst', ' - ', df['Catalyst'].unique())

Exchange  -  ['კი' 'არა']
Technical inspection  -  ['კი' 'არა']
Catalyst  -  ['კი' 'არა' nan]


In [166]:
mapper = {
    'კი': 'Yes', 'არა': 'No'
}

df['Exchange'] = df['Exchange'].map(mapper)
df['Technical inspection'] = df['Technical inspection'].map(mapper)
df['Catalyst'] = df['Catalyst'].map(mapper)

In [172]:
df.to_csv('cars-eng.csv')